In [ ]:
"""
When the domehunter package tries to import the sn3218 library it will either find it isn't installed, 
or it wont detect the hardware it is expecting in both cases a warning will be raised. If you are 
testing without the automationHAT this warning can be ignored.
"""

from domehunter.dome_control import Dome
import astropy.units as u
from astropy.coordinates import Angle, Longitude
import time

In [ ]:
# set the loggin level
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
#logger.setLevel(logging.DEBUG)

In [ ]:
"""
testing=True means all the automationHAT functionality and the state of the GPIOzero pins will be 
mocked/simulated debug_lights=True means the automationHAT status LEDs will be enabled on the 
automationHAT. If you do not have an automationHAT this should be set to False.

First positional argument is the azimuth of the home position.

NB at the moment if you try and create Dome twice it wont work because the gpio pins from the 
first instance wont be released.
"""

testdome = Dome(0, testing=True, debug_lights=False)

In [ ]:
"""
The calibrate method tells the dome to rotate n times (default n=2) and use the encoder counts to 
determine the degrees of rotation per encoder tick.
In testing mode, we will simulate 10 ticks per rotation, for 20 ticks total.
"""

testdome.calibrate_dome_encoder_counts()

In [ ]:
"""
We can now check the the degrees per tick factor and the encoder count
"""

print(f'Dome rotates {testdome.degrees_per_tick} degrees per encoder tick. Current encoder count is {testdome.encoder_count}.')

In [ ]:
"""
If we are in testing mode, lets now tell it that is at an azimuth of 90 degrees, an encoder count 
of 9 and that it rotates 10 degrees per tick. ND these are all private member variables, so to 
access them we need to use the "name mangled" member variable names
"""

testdome._encoder_count = 9
testdome._degrees_per_tick = Angle(10 * u.deg)
testdome._dome_az = Angle(90 * u.deg)

In [ ]:
print(testdome.encoder_count)
print(testdome.degrees_per_tick)
print(testdome.dome_az)
print(testdome.az_position_tolerance)

In [ ]:
"""Check where the dome thinks it is"""

testdome.dome_az

In [ ]:
"""Now we can tell it to go to an azimuth of 300 degrees. The dome will realise it is quicker to rotate anticlockwise"""

testdome.goto_az(300)
while testdome.movement_thread_active:
    time.sleep(1)

In [ ]:
print(testdome.encoder_count)
print(testdome.degrees_per_tick)
print(testdome.dome_az)
print(testdome.az_position_tolerance)

In [ ]:
"""We can now check if the dome ended up where we wanted."""

print(f'Dome is currently at an azimuth of {testdome.dome_az}, with an encoder count of {testdome.encoder_count}')

In [ ]:
"""
Currently the dome will overshoot the position depending on how fine the az_per_tick instance 
variable is (10 degrees is pretty coarse). The dome azimuth is only updated according to how 
many ticks were recorded, so even if it overshoots it should still know where it is. After 
every movement, once the dome_az is update the encoder is set to the corresponding number of 
ticks as if it had just rotated from azimuth of zero to the current location 
(encoder_count = dome_az/az_per_tick)

Now send the dome to an azimuth of 2 degrees, in this case the dome will decide to rotate clockwise.
"""

testdome.goto_az(2)
while testdome.movement_thread_active:
    time.sleep(1)

In [ ]:
"""We can now check if the dome ended up where we wanted."""

print(f'Dome is currently at an azimuth of {testdome.dome_az}, with an encoder count of {testdome.encoder_count}')